In [ ]:
import torch

from src.load_dataset import PanopticDownloader, PanopticScene
from src.panoptic_dataset import PanopticDataset

In [2]:
torch.__version__

'2.6.0+cu126'

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [4]:
# d = PanopticDownloader(device='cuda')
# # d._get_scene_names = lambda _: ['161029_hands2']  # TODO
# await d.load(scene_names_file='panoptic_scene_names.txt')

In [5]:
# d.download_views('./scenes/', 'hd', 4, 3)

In [6]:
d = PanopticDataset('./scenes')

In [7]:
d.data

[['170307_dance3',
  [{'K': [[1633.51, 0, 947.691], [0, 1629.51, 571.838], [0, 0, 1]],
    'R': [[-0.5295805751, -0.01150942367, 0.8481815535],
     [0.6366490414, 0.6553762361, 0.4063988032],
     [-0.56055545, 0.7552148849, -0.3397470605]],
    'filename': './scenes/170307_dance3/hd_00_04.mp4',
    'fps': 29.97,
    'shape': [9211, 3, 1080, 1920],
    't': [[-5.652077537], [81.64650446], [378.2933504]],
    'type': 'hd',
    'url': 'http://domedb.perception.cs.cmu.edu/webdata/dataset/170307_dance3/videos/hd_shared_crf20/hd_00_04.mp4'},
   {'K': [[1861.42, 0, 935.575], [0, 1856.72, 553.241], [0, 0, 1]],
    'R': [[0.979300228, 0.02227476313, 0.2011837426],
     [0.03031339467, 0.9665779971, -0.2545742989],
     [-0.2001303612, 0.2554032312, 0.9458948293]],
    'filename': './scenes/170307_dance3/hd_00_27.mp4',
    'fps': 29.97,
    'shape': [9211, 3, 1080, 1920],
    't': [[-0.5190184928], [123.2894969], [311.5216895]],
    'type': 'hd',
    'url': 'http://domedb.perception.cs.cmu.edu

In [8]:
v = d.__getitem__(0)
v2 = d.__getitem__(1)

In [9]:
v[0][0]._num_frames, v2[0][0]._num_frames

(9211, 11598)

In [10]:
v[0][0][-1].shape, v2[0][0][-1].shape

(torch.Size([3, 1080, 1920]), torch.Size([3, 1080, 1920]))

In [11]:
def compute_origin(v):
    K, R, t = torch.tensor(v[1]), torch.tensor(v[2]), torch.tensor(v[3])
    o = -R.T @ t
    # d = R.T @ K.inverse() @ x
    return o


In [12]:
a = [compute_origin(i) for i in v]
a2 = [compute_origin(i) for i in v2]
a, a2

# Scenes:
# 170307_dance3, hd videos 4, 27, 24, 0
# 170404_haggling_a2, hd videos 7, 0, 14, 5

([tensor([[ 157.0810],
          [-339.2670],
          [ 100.1370]]),
  tensor([[  59.1159],
          [-198.7210],
          [-263.1760]]),
  tensor([[-255.5030],
          [-119.3360],
          [  98.2647]]),
  tensor([[-272.2060],
          [-151.6980],
          [ -21.8477]])],
 [tensor([[  75.8885],
          [-312.0810],
          [ 197.1940]]),
  tensor([[-271.4690],
          [-151.7310],
          [ -21.6585]]),
  tensor([[ 151.9930],
          [-120.3060],
          [-232.8210]]),
  tensor([[-105.1600],
          [-152.5280],
          [ 254.7600]])])

From this, looking forward from the door of the dome towards the other side of it, we can find the conventions for the axes:
- x is the right axis, with the origin in the dome center;
- y is the down axis, with the origin in the ground;
- z is the forward axis, with the origin in the dome center.
